### Business Problem

A distributor of sea food to chinese restaurants wants to open an office in New york (Manhattan).  
Where would your recommend that they setup their office.
Target Audience is the Distributor of perishable sea food who wants to be close to his customers and want to do delivery to his customer on an as needed basis

### Data

We will use the data from internet which is readily available.

For the New York data I am using the same data source given to us in the previous project. Data from - https://geo.nyu.edu/catalog/nyu_2451_34572 which is downloaded and placed at my GIT location - https://github.com/msac22/Coursera_Capstone/blob/master/nyu_2451_34572-geojson.json

New York Neighborhood has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood. 

For this capstone we will just analyse Manhattan.

For the data about Chinese restaurants in each neighborhoods I am going to use the four square data.


### Methodology 

### 1. Download and get the Data

In [1]:
import pandas as pd
import requests

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import numpy as np # library to handle data in a vectorized manner
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


# import k-means from clustering stage
from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

Solving environment: \ 
  - anaconda::ca-certificates-2018.03.07-0, anaconda::openssl-1.0.2o-h26aff7b_0
  - anaconda::openssl-1.0.2o-h26aff7b_0, defaults::ca-certificates-2018.03.07-0
  - anaconda::ca-certificates-2018.03.07-0, defaults::openssl-1.0.2o-h26aff7b_0
  - defaults::ca-certificates-2018.03.07-0, defaults::openssl-1.0.2o-h26aff7bdone

# All requested packages already installed.

Solving environment: | 
  - anaconda::ca-certificates-2018.03.07-0, anaconda::openssl-1.0.2o-h26aff7b_0
  - anaconda::openssl-1.0.2o-h26aff7b_0, defaults::ca-certificates-2018.03.07-0
  - anaconda::ca-certificates-2018.03.07-0, defaults::openssl-1.0.2o-h26aff7b_0
  - defaults::ca-certificates-2018.03.07-0, defaults::openssl-1.0.2o-h26aff7bdone

# All requested packages already installed.



In [2]:
# Download and Load data 

# Download data from staged repository
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')

Data downloaded!


In [3]:
# convert Data into JSON
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
# Pull the data into a new variable from the JSON data
neighborhoods_data = newyork_data['features']

#Create Dataframe columns and instantiate the dataframe
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)

In [5]:
#loop through the dataframe and fill the dataframe
# Taken from the New York lab in coursera

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


######  Find the Data for Manhattan
Restriected it to just Manahattan because of the Four square limitation on the API calls.

In [6]:
neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
neighborhoods.shape

(40, 4)

### 2. Exploring the Data set

In [7]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 1 boroughs and 40 neighborhoods.


###### Use geopy library to get the latitude and longitude values of New York City.

In [8]:
address = 'Manhattan, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

/anaconda3/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of New York City are 40.7900869, -73.9598295.


##### Create a map of New York with Markers of neighbourhood -  Map not showing up in Git but it works in Jupter notebook

In [9]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

##### Four square Credentials

In [10]:
CLIENT_ID = 'FQPQWRZK01LVCALJ5KT51C1MARBHYM0MCEKIQAY32BCS4AG4' # your Foursquare ID
CLIENT_SECRET = 'EAOSNF00LZVG1E0QS2BMTWEE2POTU4ANYNBIODBGOQ245S2E' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

##### Exploring the first neighbourhood

In [11]:
neighborhoods.loc[0, 'Neighborhood']

'Marble Hill'

##### Get the neighborhood's latitude and longitude values.

In [12]:
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


####  Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters. 
####  The difference between what was done in the Coursera lab and here is that I use the 'Section = Food' filter from Foursquare to get that sectional data.  

In [13]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
section = 'food' # find category of food from four squeare
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&section={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT,
    section)

In [14]:
results = requests.get(url).json()

In [15]:
# function that extracts the category of the venue
# Taken from the foursquare lab in coursera
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

##### Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [16]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Arturo's,Pizza Place,40.874412,-73.910271
1,Tibbett Diner,Diner,40.880404,-73.908937
2,Dunkin' Donuts,Donut Shop,40.876993,-73.906507
3,Land & Sea Restaurant,Seafood Restaurant,40.877885,-73.905873
4,Parrilla Latina,Steakhouse,40.877473,-73.906073
5,Boston Market,American Restaurant,40.877430,-73.905412
6,SUBWAY,Sandwich Place,40.878493,-73.905385
7,Subway Sandwiches,Sandwich Place,40.874667,-73.909586
8,Subway,Sandwich Place,40.877720,-73.905380
9,Kam Wah Kitchen,Asian Restaurant,40.876084,-73.908698


In [17]:
print('{} food venue(s) were returned by Foursquare for {}'.format(nearby_venues.shape[0], neighborhoods.loc[0, 'Neighborhood']) )

16 food venue(s) were returned by Foursquare for Marble Hill


### 3. Explore Neighborhoods in New York

##### Let's create a function to repeat the same process to all the neighborhoods

In [18]:
def getNearbyVenues(names, latitudes, longitudes, section, radius=500):
    venues_list=[]
    i=1
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name,i)
        i = i + 1
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&section={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            section)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

##### Now write the code to run the above function on each neighborhood and create a new dataframe called *newyork_venues*.
#####  The difference between what was done in the Coursera lab and here is that I use the 'Section = Food' filter from Foursquare to get that sectional data.  

In [19]:
# find category of food from four squeare
newyork_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude'],
                                   section='food')

Marble Hill 1
Chinatown 2
Washington Heights 3
Inwood 4
Hamilton Heights 5
Manhattanville 6
Central Harlem 7
East Harlem 8
Upper East Side 9
Yorkville 10
Lenox Hill 11
Roosevelt Island 12
Upper West Side 13
Lincoln Square 14
Clinton 15
Midtown 16
Murray Hill 17
Chelsea 18
Greenwich Village 19
East Village 20
Lower East Side 21
Tribeca 22
Little Italy 23
Soho 24
West Village 25
Manhattan Valley 26
Morningside Heights 27
Gramercy 28
Battery Park City 29
Financial District 30
Carnegie Hill 31
Noho 32
Civic Center 33
Midtown South 34
Sutton Place 35
Turtle Bay 36
Tudor City 37
Stuyvesant Town 38
Flatiron 39
Hudson Yards 40


In [20]:
print(newyork_venues.shape)
newyork_venues.head()

(2861, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
2,Marble Hill,40.876551,-73.91066,Dunkin' Donuts,40.876993,-73.906507,Donut Shop
3,Marble Hill,40.876551,-73.91066,Land & Sea Restaurant,40.877885,-73.905873,Seafood Restaurant
4,Marble Hill,40.876551,-73.91066,Parrilla Latina,40.877473,-73.906073,Steakhouse


##### Let's check how many venues were returned for each neighborhood

In [21]:
newyork_venues.groupby('Neighborhood').size().reset_index(name='counts')

,Neighborhood,counts
0,Battery Park City,39
1,Carnegie Hill,68
2,Central Harlem,47
3,Chelsea,100
4,Chinatown,100
5,Civic Center,85
6,Clinton,100
7,East Harlem,50
8,East Village,100
9,Financial District,100


### 4. Analyze each Neighborhood in New York

In [22]:
# one hot encoding
newyork_onehot = pd.get_dummies(newyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
newyork_onehot['Neighborhood'] = newyork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [newyork_onehot.columns[-1]] + list(newyork_onehot.columns[:-1])
newyork_onehot = newyork_onehot[fixed_columns]
newyork_onehot.head()

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,BBQ Joint,...,Theme Restaurant,Tibetan Restaurant,Tonkatsu Restaurant,Turkish Restaurant,Udon Restaurant,Ukrainian Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Wings Joint
0,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
newyork_onehot.shape

(2861, 123)

###### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [24]:
newyork_grouped = newyork_onehot.groupby('Neighborhood').mean().reset_index()
newyork_grouped.shape

(40, 123)

In [25]:
newyork_grouped.shape

(40, 123)

###### Let's put that into a *pandas* dataframe

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
num_top_venues = 4

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = newyork_grouped['Neighborhood']

for ind in np.arange(newyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(newyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
0,Battery Park City,Pizza Place,Burger Joint,Italian Restaurant,Chinese Restaurant
1,Carnegie Hill,Pizza Place,Café,Bakery,Italian Restaurant
2,Central Harlem,Fried Chicken Joint,Southern / Soul Food Restaurant,Deli / Bodega,Chinese Restaurant
3,Chelsea,Bakery,Italian Restaurant,Café,Japanese Restaurant
4,Chinatown,Chinese Restaurant,Bakery,Vietnamese Restaurant,Dim Sum Restaurant
5,Civic Center,Italian Restaurant,Sandwich Place,French Restaurant,American Restaurant
6,Clinton,American Restaurant,Italian Restaurant,Sandwich Place,Deli / Bodega
7,East Harlem,Mexican Restaurant,Pizza Place,Bakery,Latin American Restaurant
8,East Village,Pizza Place,Mexican Restaurant,Japanese Restaurant,Vietnamese Restaurant
9,Financial District,Sandwich Place,American Restaurant,Italian Restaurant,Café


In [28]:
neighborhoods_venues_sorted.shape

(40, 5)

### 5. Cluster Neighborhoods

##### Run k-means to cluster the neighborhood into 5 clusters.

In [29]:
# set number of clusters
kclusters = 6

newyork_grouped_clustering = newyork_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(newyork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
newyork_grouped_clustering.shape

(40, 122)

In [30]:
newyork_grouped_clustering

,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,...,Theme Restaurant,Tibetan Restaurant,Tonkatsu Restaurant,Turkish Restaurant,Udon Restaurant,Ukrainian Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Wings Joint
0,0.00,0.00000,0.051282,0.000000,0.000000,0.000000,0.000000,0.000000,0.051282,0.000000,...,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000
1,0.00,0.00000,0.014706,0.000000,0.014706,0.000000,0.000000,0.000000,0.000000,0.014706,...,0.000000,0.00,0.00,0.014706,0.00,0.00,0.000000,0.00,0.029412,0.000000
2,0.00,0.06383,0.042553,0.000000,0.000000,0.000000,0.000000,0.000000,0.042553,0.021277,...,0.000000,0.00,0.00,0.000000,0.00,0.00,0.021277,0.00,0.000000,0.000000
3,0.00,0.00000,0.040000,0.010000,0.000000,0.030000,0.010000,0.000000,0.010000,0.020000,...,0.000000,0.00,0.00,0.000000,0.00,0.00,0.030000,0.00,0.030000,0.000000
4,0.00,0.00000,0.030000,0.000000,0.000000,0.020000,0.010000,0.010000,0.000000,0.010000,...,0.000000,0.00,0.00,0.000000,0.00,0.00,0.030000,0.00,0.060000,0.000000
5,0.00,0.00000,0.058824,0.000000,0.000000,0.023529,0.011765,0.000000,0.000000,0.011765,...,0.011765,0.00,0.00,0.000000,0.00,0.00,0.011765,0.00,0.011765,0.011765
6,0.00,0.00000,0.080000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.000000,0.00,0.00,0.020000,0.00,0.000000,0.000000
7,0.00,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000
8,0.00,0.00000,0.020000,0.020000,0.010000,0.010000,0.000000,0.000000,0.010000,0.020000,...,0.000000,0.00,0.00,0.000000,0.01,0.01,0.040000,0.00,0.050000,0.010000
9,0.00,0.00000,0.070000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.010000,...,0.000000,0.00,0.00,0.000000,0.00,0.00,0.030000,0.00,0.000000,0.000000


#### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [31]:
newyork_merged = neighborhoods
newyork_merged.shape
# add clustering labels
newyork_merged['Cluster Labels'] = kmeans.labels_

# merge newyork_grouped with newyork_data to add latitude/longitude for each neighborhood
newyork_merged = newyork_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

newyork_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,2,Sandwich Place,Deli / Bodega,Pizza Place,Steakhouse
1,Manhattan,Chinatown,40.715618,-73.994279,2,Chinese Restaurant,Bakery,Vietnamese Restaurant,Dim Sum Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,4,Pizza Place,Chinese Restaurant,Bakery,Deli / Bodega
3,Manhattan,Inwood,40.867684,-73.921210,1,Deli / Bodega,Mexican Restaurant,Pizza Place,Bakery
4,Manhattan,Hamilton Heights,40.823604,-73.949688,4,Deli / Bodega,Mexican Restaurant,Chinese Restaurant,Pizza Place


##### Finally, let's visualize the resulting clusters -  Map not showing up in Git but it works in Jupter notebook

In [32]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(newyork_merged['Latitude'], newyork_merged['Longitude'], newyork_merged['Neighborhood'], newyork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 6. Examine Clusters

###### Cluster 1 - Red in above Map

In [33]:
newyork_merged.loc[newyork_merged['Cluster Labels'] == 0, 
                   newyork_merged.columns[[1] + 
                   list(range(5, newyork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
24,West Village,Italian Restaurant,American Restaurant,New American Restaurant,Gastropub


###### Cluster 2  - Purple in above map

In [34]:
newyork_merged.loc[newyork_merged['Cluster Labels'] == 1, 
                   newyork_merged.columns[[1] + 
                   list(range(5, newyork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
3,Inwood,Deli / Bodega,Mexican Restaurant,Pizza Place,Bakery
5,Manhattanville,Deli / Bodega,Chinese Restaurant,Seafood Restaurant,Mexican Restaurant
10,Lenox Hill,Italian Restaurant,Sushi Restaurant,Pizza Place,Deli / Bodega
12,Upper West Side,Italian Restaurant,Indian Restaurant,French Restaurant,Vegetarian / Vegan Restaurant
16,Murray Hill,Sandwich Place,American Restaurant,Japanese Restaurant,Pizza Place
18,Greenwich Village,Italian Restaurant,Café,Sushi Restaurant,American Restaurant
27,Gramercy,Italian Restaurant,Deli / Bodega,Restaurant,Mexican Restaurant
29,Financial District,Sandwich Place,American Restaurant,Italian Restaurant,Café
32,Civic Center,Italian Restaurant,Sandwich Place,French Restaurant,American Restaurant
35,Turtle Bay,Italian Restaurant,Café,Deli / Bodega,Sushi Restaurant


###### Cluster 3 - Blue in above map

In [35]:
newyork_merged.loc[newyork_merged['Cluster Labels'] == 2, 
                   newyork_merged.columns[[1] + 
                   list(range(5, newyork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
0,Marble Hill,Sandwich Place,Deli / Bodega,Pizza Place,Steakhouse
1,Chinatown,Chinese Restaurant,Bakery,Vietnamese Restaurant,Dim Sum Restaurant
6,Central Harlem,Fried Chicken Joint,Southern / Soul Food Restaurant,Deli / Bodega,Chinese Restaurant
8,Upper East Side,Italian Restaurant,American Restaurant,Diner,Pizza Place
9,Yorkville,Pizza Place,Italian Restaurant,Deli / Bodega,Sandwich Place
11,Roosevelt Island,Deli / Bodega,Sandwich Place,Café,Japanese Restaurant
20,Lower East Side,Pizza Place,Chinese Restaurant,Deli / Bodega,Bakery
23,Soho,Italian Restaurant,French Restaurant,Café,American Restaurant
26,Morningside Heights,Deli / Bodega,Food Truck,Café,Pizza Place
31,Noho,Italian Restaurant,Pizza Place,Mexican Restaurant,Sushi Restaurant


###### Cluster 4 -  green in above map

In [36]:
newyork_merged.loc[newyork_merged['Cluster Labels'] == 3, 
                   newyork_merged.columns[[1] + 
                   list(range(5, newyork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
14,Clinton,American Restaurant,Italian Restaurant,Sandwich Place,Deli / Bodega
17,Chelsea,Bakery,Italian Restaurant,Café,Japanese Restaurant
25,Manhattan Valley,Indian Restaurant,Chinese Restaurant,Pizza Place,Deli / Bodega


###### Cluster 5 - light green in above map

In [37]:
newyork_merged.loc[newyork_merged['Cluster Labels'] == 4, 
                   newyork_merged.columns[[1] + 
                   list(range(5, newyork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
2,Washington Heights,Pizza Place,Chinese Restaurant,Bakery,Deli / Bodega
4,Hamilton Heights,Deli / Bodega,Mexican Restaurant,Chinese Restaurant,Pizza Place
7,East Harlem,Mexican Restaurant,Pizza Place,Bakery,Latin American Restaurant
13,Lincoln Square,Café,Food Truck,Italian Restaurant,American Restaurant
15,Midtown,American Restaurant,Food Truck,Sandwich Place,Bakery
19,East Village,Pizza Place,Mexican Restaurant,Japanese Restaurant,Vietnamese Restaurant
21,Tribeca,American Restaurant,Italian Restaurant,Café,Sandwich Place
22,Little Italy,Italian Restaurant,Bakery,Chinese Restaurant,Café
28,Battery Park City,Pizza Place,Burger Joint,Italian Restaurant,Chinese Restaurant
37,Stuyvesant Town,Sandwich Place,German Restaurant,Deli / Bodega,Indian Restaurant


###### Cluster 6 - orange in above map

In [38]:
newyork_merged.loc[newyork_merged['Cluster Labels'] == 5, 
                   newyork_merged.columns[[1] + 
                   list(range(5, newyork_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
30,Carnegie Hill,Pizza Place,Café,Bakery,Italian Restaurant


### Results

I have to change the Four square API calls to just focus on 'section = food' from the original coursera example.  This gave us more targetted results since our business problem is focused on the food industry.  I also tried to use the search query in Four square to check the total restaurants in an area. The free query only gives a limited response so I could not join the counts data with this data.  Ideally it will be nice to get that specific category data for the specific location and combine it.

Initially, having the radius for the Foursquare API at section of food and 1000 meters yielded some over lap and the clusters became homogeneous and over lapping.

750 meters also yielded some over lap and made the clusters more homogeneous.  Decreasing it to 500 meters yielded the most optimal results.

I also tried working with different clusters.  Started with cluster of 3 and that gave more broader results with no specifics.  The ideal one seems to be a cluster of 6.

Superficially looking at the clusters we have an outlier to the very north. East and south side of the city has more restaurants than the west and north side.

##### Cluster 1
Italian restaurant as most common.

##### Cluster 2
Itallian restaurant and deli places as most common.

##### Cluster 3
Has Chinese restaurant as the top and second choice.

##### Cluster 4
Has American and Indian restaurant

##### Cluster 5
Has more wide spread first choices and distibuted

##### Cluster 6
Has pizza as most common

### Discussion



With the insights obtained through Methodology and Results we can further label the clusters based off of their inherent common venues.
##### Cluster 1
Primarily niche areas which does not have that many restaurants (west side of down town area).


##### Cluster 2
Italian Restaurants and deli venues that are scattered around Manhattan.


##### Cluster 3
Concentrated in two areas.  One in the lower east side and another in the upper west side.  
Looking furthur into this area the lower east side with China town has more concentration of Chinese restaurant


##### Cluster 4
More American and Indian Restaurants focused around the west side of Manhattan.

##### Cluster 5
Looks like mostly lunch crowd with American Deli and food truck choices.

##### Cluster 6
Outskirt neightbour hood with Pizza choice.

With these insights on each cluster we can create a more coherent statement about Manhattan Restaurant pattern.

Manhattan is a city with distinct east and west halves dominated by different kind of cultural restaurants.  It is a melting pot of different cultures with restaurants ranging from Chinese to Korean to American.  It clear from the clusters that each neighbourhood is popular for a  unique speciality of restaurants.  


### Conclusion

Based on the results above it is clear that the distributor should setup his office close to cluster 3 primarily close to China town and Lower East side.  They can expand to the adjacent Midtown south neightbourhood which is also in the same cluster 3.  To extend this report we can further analysis on the number of Chinese restaurant and the revenue of the location.  We can also look at the cost of rental in this area to see if the distributor can get an ideal location for the cost. As a secondary report I tried to also look at Brooklyn (since it is across the bridge from cluster 3) restaurant patterns.

The final conclusion is as I expected the distributor should setup his office close to Lower East and China town and they can expand into Midtown south and also to Brooklyn.
